# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
pip install sklearn-crfsuite

Note: you may need to restart the kernel to use updated packages.


#### Import the necessary libraries

In [2]:
import nltk
import joblib
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(input_sigs, output_labels):
    # Initialize an empty list to store the tuples for each sentence
    tuples_list = []
    
    # Iterate over each sentence (list of tokens) and its corresponding labels
    for tokens, labels in zip(input_sigs, output_labels):
        # Create tuples for each token-label pair and append to the list
        tuples = [(token, label) for token, label in zip(tokens, labels)]
        tuples_list.append(tuples)
    
    return tuples_list

In [6]:
tuples_maker(input_sigs, output_labels)

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [7]:
nltk.download('averaged_perceptron_tagger')

def triples_maker(tuples_list):
    # Initialize an empty list to store the triples for each sentence
    triples_list = []
    
    # Iterate over each sentence (list of tuples)
    for tuples in tuples_list:
        # Use NLTK's pos_tag to obtain POS tags for tokens
        pos_tags = nltk.pos_tag([token for token, _ in tuples])
        
        # Create triples by combining token, POS tag, and label
        triples = [(token, pos_tag, label) for (token, label), (_, pos_tag) in zip(tuples, pos_tags)]
        
        # Append the triples to the list
        triples_list.append(triples)
    
    return triples_list

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
sample_data = triples_maker(tuples_maker(input_sigs, output_labels))
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [9]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [10]:
output_labels

[['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'],
 ['Method', 'Qty', 'Form'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['EVERY', 'Period'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['Method', 'Qty', 'TO', 'Qty', 'Form'],
 ['Method', 'Qty', 'TO', 'Qty', 'Form'],
 ['Method',
  'Qty',
  'Form',
  'PO',
  'BID',
  'FOR',
  'Duration',
  'DurationUnit',
  'AT',
  'WHEN'],
 ['Method', 'Qty', 'Form', 'TID', 'PO'],
 ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'],
 ['Method',
  'Qty',
  'Form',
  'BY',
  'PO',
  'TID',
  'FOR',
  'Duration',
  'DurationUnit'],
 ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'],
 ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'],
 

In [11]:
def extract_features(doc):
    return [token_to_features(doc, i) for i in range(len(doc))]

X = [extract_features(doc) for doc in sample_data]

In [12]:
y = [[label for _, _, label in sentence] for sentence in sample_data]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Training the CRF model with the features extracted using the feature extractor method

In [14]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)

In [15]:
crf.fit(X_train, y_train)
joblib.dump(crf, 'crf_model.pkl')

loading training data to CRFsuite: 100%|████████████████████████████████████████████| 44/44 [00:00<00:00, 11260.56it/s]


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1784
Seconds required: 0.004

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.00  loss=587.40   active=1780  feature_norm=1.00
Iter 2   time=0.00  loss=367.72   active=1558  feature_norm=5.30
Iter 3   time=0.00  loss=271.70   active=1559  feature_norm=6.63
Iter 4   time=0.00  loss=233.49   active=1367  feature_norm=7.08
Iter 5   time=0.00  loss=169.42   active=1182  feature_norm=8.62
Iter 6   time=0.00  loss=125.81   active=1103  feature_norm=10.30
Iter 7   time=0.00  loss=102.98   active=1001  feature_norm=14.31
Iter 8   time=0.00  loss=83.00    active=1086  feature_norm=14.70
Iter 9   time=0.00  loss=79.87    active=1082  feature_norm=14.59
Iter 10  ti

['crf_model.pkl']

### Predicting the test data with the built model

In [16]:
y_pred = crf.predict(X_test)

In [17]:
y_pred

[['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['Method', 'Qty', 'Form'],
 ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['QID', 'PeriodUnit'],
 ['QID'],
 ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'],
 ['Qty',
  'Form',
  'FOR',
  'Duration',
  'DurationUnit',
  'Qty',
  'TIMES',
  'Period',
  'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['EVERY', 'Period'],
 ['Method', 'DAILY'],
 ['Q46H']]

### Putting all the prediction logic inside a predict method

In [18]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    
    words = nltk.word_tokenize(sig)
    doc = nltk.pos_tag(words)
    doc_features = extract_features(doc)
    predicted_labels = crf.predict([doc_features])[0]
    
    return predicted_labels

### Sample predictions

In [19]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
predictions

['Method',
 'Qty',
 'Form',
 'EVERY',
 'Period',
 'PeriodUnit',
 'FOR',
 'Duration',
 'DurationUnit']

In [20]:
predictions = predict("2 capsu for 10 day at bed")
predictions

['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']

In [21]:
predictions = predict("2 capsu for 10 days at bed")
predictions

['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']

In [22]:
predictions = predict("5 days 2 tabs at bed")
predictions

['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']

In [23]:
predictions = predict("3 tabs qid x 10 weeks")

In [24]:
predictions = predict("x 30 days")
predictions

['FOR', 'Duration', 'DurationUnit']

In [25]:
predictions = predict("x 20 months")
predictions

['FOR', 'Duration', 'DurationUnit']

In [26]:
predictions = predict("take 2 tabs po tid for 10 days")
predictions

['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']

In [27]:
predictions = predict("take 2 capsules po every 6 hours")
predictions

['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']

In [28]:
predictions = predict("inject 2 units pu tid")
predictions

['Method', 'Qty', 'Form', 'PO', 'TID']

In [29]:
predictions = predict("swallow 3 caps tid by mouth")
predictions

['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']

In [30]:
predictions = predict("inject 3 units orally")
predictions

['Method', 'Qty', 'Form', 'PO']

In [31]:
predictions = predict("orally take 3 tabs tid")
predictions

['PO', 'Method', 'Qty', 'Form', 'TID']

In [32]:
predictions = predict("by mouth take three caps")
predictions

['BY', 'PO', 'Method', 'Qty', 'Form']

In [33]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
predictions

['Method',
 'Qty',
 'Form',
 'PO',
 'Qty',
 'TIMES',
 'Period',
 'PeriodUnit',
 'FOR',
 'Duration',
 'DurationUnit',
 'AT',
 'WHEN']

In [34]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")
predictions

['Method',
 'Qty',
 'Form',
 'PO',
 'BID',
 'FOR',
 'Duration',
 'DurationUnit',
 'AT',
 'WHEN']

In [35]:
predictions = predict("take 3 tabs bid orally at bed")
predictions

['Method', 'Qty', 'Form', 'TID', 'PO', 'AT', 'WHEN']

In [36]:
predictions = predict("take 10 capsules by mouth qid")
predictions

['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']

In [37]:
predictions = predict("inject 10 units orally qid x 3 months")
predictions

['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']

In [38]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")
prediction

['Method',
 'Method',
 'Qty',
 'Form',
 'Frequency',
 'PeriodUnit',
 'FOR',
 'Period',
 'PeriodUnit',
 'EVERY',
 'Period',
 'PeriodUnit',
 'AND',
 'EVERY',
 'Period',
 'PeriodUnit']

In [39]:
prediction = predict("Amoxcicillin QID 30 tablets")
prediction

['Qty', 'Method', 'Qty', 'Form']

In [40]:
prediction = predict("take 3 tabs TID for 90 days with food")
prediction

['Method',
 'Qty',
 'Form',
 'Frequency',
 'FOR',
 'Duration',
 'DurationUnit',
 'WITH',
 'FOOD']

In [41]:
prediction = predict("with food take 3 tablets per day for 90 days")
prediction

['WITH',
 'FOOD',
 'Method',
 'Qty',
 'Form',
 'Frequency',
 'PeriodUnit',
 'FOR',
 'Duration',
 'DurationUnit']

In [42]:
prediction = predict("with food take 3 tablets per week for 90 weeks")
prediction

['WITH',
 'FOOD',
 'Method',
 'Qty',
 'Form',
 'Frequency',
 'PeriodUnit',
 'FOR',
 'Duration',
 'DurationUnit']

In [43]:
prediction = predict("take 2-4 tabs")
prediction

['Method', 'Qty', 'Form']

In [44]:
prediction = predict("take 2 to 4 tabs")
prediction

['Method', 'Qty', 'TO', 'Qty', 'Form']

In [45]:
prediction = predict("take two to four tabs")
prediction

['Method', 'Qty', 'TO', 'Qty', 'Form']

In [46]:
prediction = predict("take 2-4 tabs for 8 to 9 days")
prediction

['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']

In [47]:
prediction = predict("take 20 tabs every 6 to 8 days")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [48]:
prediction = predict("take 2 tabs every 4 to 6 days")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [49]:
prediction = predict("take 2 tabs every 2 to 10 weeks")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [50]:
prediction = predict("take 2 tabs every 4 to 6 days")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [51]:
prediction = predict("take 2 tabs every 2 to 10 months")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [52]:
prediction = predict("every 60 mins")
prediction

['EVERY', 'Period', 'PeriodUnit']

In [53]:
prediction = predict("every 10 mins")
prediction

['EVERY', 'Period', 'PeriodUnit']

In [54]:
predictionsprediction = predict("every two to four months")
prediction

['EVERY', 'Period', 'PeriodUnit']

In [55]:
prediction = predict("take 2 tabs every 3 to 4 days")
prediction

['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']

In [56]:
prediction = predict("every 3 to 4 days take 20 tabs")
prediction

['EVERY', 'Period', 'TO', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form']

In [57]:
prediction = predict("once in every 3 days take 3 tabs")
prediction

['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']

In [58]:
prediction = predict("take 3 tabs once in every 3 days")
prediction

['Method',
 'Qty',
 'Form',
 'Frequency',
 'Period',
 'EVERY',
 'Period',
 'PeriodUnit']

In [59]:
prediction = predict("orally take 20 tabs every 4-6 weeks")
prediction

['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']

In [60]:
prediction = predict("10 tabs x 2 days")
prediction

['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']

In [61]:
prediction = predict("3 capsule x 15 days")
prediction

['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']

In [62]:
prediction = predict("10 tabs")
prediction

['Qty', 'Form']